# Tribbles

some terrible traitlets tricks

> **Don't** consider this **supported code**, but rather an example of operator overloading with widgets. 

In [ ]:
import ipywidgets as W, traitlets as T, IPython as I
from contextlib import contextmanager
from functools import lru_cache

In [ ]:
def _make_dlink_(fn, *args):
    def _dlink_(val):
        return fn(*[
            ~arg for arg in args
        ])
    return _dlink_

In [ ]:
class Bit():
    def __init__(self, has_traits, name):
        self._has_traits_ = has_traits
        self._name_ = name
    def __invert__(self): return getattr(self._has_traits_, self._name_)
    def __eq__(self, other): T.link((self._has_traits_, self._name_), (other._has_traits_, self._name_))
    def __le__(self, other): 
        if isinstance(other, tuple):
            fn = other[0]
            for inst in other[1:]:
                T.dlink(
                    (inst._has_traits_, inst._name_), 
                    (self._has_traits_, self._name_),
                    _make_dlink_(*other)
                )
        else:
            T.dlink(
                (other._has_traits_, other._name_),
                (self._has_traits_, self._name_),
            )

In [ ]:
class Tidbit():
    _has_traits_ = None
    
    def __init__(self, has_traits: T.HasTraits):
        self._has_traits_ = has_traits
        
    @lru_cache
    def __getattr__(self, name): return Bit(self._has_traits_, name)

In [ ]:
class Tribble():
    def __init__(self, container, **items): 
        self._trible_container_ = container
        container.children = [*items.values()]
        self._trible_ns_ = {k: Tidbit(i) for k, i in items.items()}
        
    @lru_cache
    def __getattr__(self, attr): return self._trible_ns_[attr]

    @lru_cache
    def __getitem__(self, item): return Tidbit(item)
    
    def ui(self):
        return self._trible_container_

A _tribble_ is a symbolic namespace for the _traitlets_ underneath _widgets_. Consider a simple box and container.

In [ ]:
a = W.FloatSlider(description="a")
box = W.HBox([a])

It accepts a single box-like container, and a keyword arguments of named widgets.

In [ ]:
tv = Tribble(box, a=a, **{i: W.FloatSlider(description=i) for i in "bcd"})
tv.ui()

Access widgets by name, or as an index:

In [ ]:
print(tv[a].value, tv.a.value)

To actually access the value of a wrapped widget, use the `~` prefix operator.

In [ ]:
a.value = 10
~tv.a.value

Link two traits together with the equality operator `==`.

In [ ]:
tv.a.value == tv.b.value
~tv.b.value

You can use these inside trait handlers as "slow" values, that are _used_ during a functional transformation, but should _not_ trigger recalculation.

In [ ]:
tv.d.value <= (lambda c: c * ~tv.b.value / 10, 
               tv.c.value)

Any number of "fast" arguments can be provided.

In [ ]:
tv.c.value <= (lambda b, d: b + d,
               tv.b.value,
               tv.d.value)

Other `Box` subclasses work as well.

In [ ]:
from wxyz.lab import DockBox
import math, random
dtv = Tribble(DockBox(layout=dict(height="300px")), **{i: W.FloatSlider(description=i, value=random.random() * 100) for i in "wxyz"})
dtv.z.value <= (lambda w, x, y: (math.sin(w) * x) + y,
               dtv.w.value,
               dtv.x.value,
               dtv.y.value)
dtv.ui()